In [115]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [116]:
df=pd.read_csv("internship_dataset_extended_corrected.csv")
df.head()

,Internship_ID,Company,Role,Sector,Location,Stipend(INR),Duration(Months),Required_Skills,Mode,Openings,Education_Requirement,Experience_Level
0,INT0001,Microsoft,Software Engineer Intern,Software Development,Chennai,10000,2,"Data Structures, C++",Onsite,2,Bachelor,Fresher
1,INT0002,Google,AI Research Intern,AI/ML,Kolkata,10000,5,"PyTorch, NLP",Onsite,9,Bachelor,Fresher
2,INT0003,TCS,Web Developer Intern,Web Development,Chennai,8000,3,"Bootstrap, Node.js, REST APIs, HTML/CSS/JS",Onsite,7,Bachelor,Fresher
3,INT0004,Google,Data Analyst Intern,Data Science,Mumbai,5000,5,"Pandas, Tableau, Python, Statistics",Onsite,10,Bachelor,Fresher
4,INT0005,TCS,AI Research Intern,AI/ML,Hyderabad,15000,3,"Python, Machine Learning, NLP",Onsite,6,Bachelor,Fresher


In [125]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Load dataset
df = pd.read_csv("internship_dataset_extended_corrected.csv")

# Weighted text: Sector > Education > Skills > Location
df["Combined_Text"] = (
    (df["Sector"].astype(str) + " ") * 3 +       # Highest priority
    (df["Education_Requirement"].astype(str) + " ") * 2 +  # Second priority
    (df["Required_Skills"].astype(str) + " ") * 4 +        # Third priority
    df["Location"].astype(str)                              # Lowest priority
)

# Vectorize combined features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["Combined_Text"])

# Train KMeans
kmeans = KMeans(n_clusters=10, random_state=42)
df["Cluster"] = kmeans.fit_predict(X)

# Recommendation function
def recommend_internships(skills, sector, location, education, n=3):
    student_input = (
        (sector + " ") * 3 +
        (education + " ") * 2+
        (skills + " ") * 4 +
        location
    )
    student_vec = vectorizer.transform([student_input])
    
    cluster = kmeans.predict(student_vec)[0]
    recs = df[df["Cluster"] == cluster].sample(n)
    return recs[[
        "Internship_ID", "Company", "Role", "Location", 
        "Required_Skills", "Sector", "Education_Requirement", 
        "Stipend(INR)"
    ]]

# Example usage
result=pd.DataFrame(recommend_internships(
    skills="Python, Machine Learning, SQL",
    sector="Data Science",
    location="Bangalore",
    education="Bachelor",
    n=5
))
result


,Internship_ID,Company,Role,Location,Required_Skills,Sector,Education_Requirement,Stipend(INR)
239,INT0240,Oracle,ML Engineer Intern,Mumbai,"Python, Machine Learning, TensorFlow",AI/ML,Master,20000
289,INT0290,Accenture,AI Research Intern,Hyderabad,"Machine Learning, TensorFlow, PyTorch",AI/ML,Bachelor,20000
1398,INT1399,Microsoft,ML Engineer Intern,Pune,"Python, Machine Learning",AI/ML,Bachelor,15000
38,INT0039,Oracle,ML Engineer Intern,Mumbai,"PyTorch, Machine Learning",AI/ML,Diploma,10000
377,INT0378,Microsoft,AI Research Intern,Pune,"Python, Machine Learning, NLP, PyTorch",AI/ML,Bachelor,15000
